# FIRST QUICK TOOL TO TEST OPENAI API CONNECTION
### Create a tool that summarizes an online article or blog post.

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! It’s great to hear from you. How can I assist you today?


## Scrape the website

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Let's try one out. Change the website and add print statements to follow along.

post = Website("https://zandersgroup.com/en/ifrs9-annual-report-study")
print(post.title)
print("------")
print(post.text)

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [8]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of an article \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at an article titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
system_prompt

'You are an assistant that analyzes the contents of an article and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'

In [ ]:
print(user_prompt_for(post))

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [12]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [13]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, you're really hitting me with the tough questions, aren't you? But fine, I’ll humor you. 2 + 2 equals 4. Shocking, I know!


## And now let's build useful messages for GPT-4o-mini, using a function

In [14]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
# Try this out, and then try for a few more websites

messages_for(post)

## Time to bring it together - the API for OpenAI is very simple!

In [16]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://zandersgroup.com/en/ifrs9-annual-report-study")

In [18]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://zandersgroup.com/en/ifrs9-annual-report-study")

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(

## Now with open source ollama
We already have the prompts and messages list.
Only need to get the response via ollama.

In [ ]:
# pull and specify the model - we try llama3.2:1b because llama3.2 runs very slow on the lappy
!ollama pull llama3.2:1b

In [27]:
MODEL = "llama3.2:1b"

In [ ]:
url = "https://zandersgroup.com/en/ifrs9-annual-report-study"
website = Website(url)
messages = messages_for(website)
messages

In [29]:
# import ollama and get response via allama library
import ollama

response = ollama.chat(model="llama3.2:1b", messages=messages)
print(response['message']['content'])

The article appears to be an announcement or a press release from Zanders Group, a consulting firm that specializes in treasury and risk management solutions for financial institutions.

Here are the key points highlighted in the text:

1. **Regulatory updates**: The article mentions changes in regulations related to Credit Risk Regulation (CRR) and External Review Arrangements (ERA) under the Basel III Directive.
2. **BCBS 239's RDARR principles**: Zanders highlights the importance of effective risk data aggregation and reporting (RDARR) as a key vulnerability in its planning of supervisory priorities for the banking sector.
3. **Mastering pre-hedge strategies**: The article discusses the need for financial institutions to master pre-hedge strategies, including data-driven FX and risk management, to achieve success in 2025.

The text also mentions some upcoming events or webinars hosted by Zanders experts, such as a webinar on Mastering Pre-Hedge Strategies and another on "Propel your

In [30]:
# another option is to get it via the openAI library
# for some reason you can call ollama via the openAI library as well
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

The article appears to be from an unknown or lesser-known news source, but it seems to be related to banking and finance. The content includes several mentions of companies and organizations, such as:

*  Zanders Group: a financial consulting firm that provides advisory services in areas like Treasury, Risk Management, and Compliance.
* FIntegral: a company that specializes in risk management software
* Optimum Prime: another company that provides consulting services in the same industries

Some specific mentions of notable institutions include:

* Basel Committee on Banking Supervision (BCBS): an international watchdog organization responsible for setting banking regulations globally.
* European Central Bank (ECB): a central bank that controls monetary policy and regulates banks in Europe.
* Comisión Nacional de Seguros y Fomento para la Economía (COFEMA): the Mexican government's insurance regulator.

However, none of this information seems to be directly related to the content menti

In [33]:
# Now let's get it via deepseek
# !ollama pull deepseek-r1:1.5b - done
response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=messages
)

print(response.choices[0].message.content)

<think>
Okay, so I'm trying to figure out how to manage this Zanders acquisition of Fintegral, RiskQuest, and Optimum Prime. Let's see... 

First off, the user wants to understand the reasons behind these acquisitions. I guess it has something to do with improving customer experience or perhaps their capabilities? They mentioned risk management tools like Risk Quest and Optimum Prime, which makes sense because those are things companies use to manage financial risks better.

So, when Zanders acquired Fintegral, RQ, and OP along with them, that must have been a strategic move. Maybe they wanted to centralize all these tools under one roof for their customers. That would make managing their risk solutions easier for employees who might feel overwhelmed if they have several platforms. Oh, I wonder how it was managed once—when did Fintegral become a service on the main platform?

I'm also thinking about the performance and benefits gained from these acquisitions. Did they see an increase i